In [ ]:
import pandas as pd
import json
from sklearn.cluster import KMeans
import numpy as np

import sys
import os

sys.path.append(os.path.abspath('../'))
from tool import *

## Tool functions

In [3]:
# import yfinance as yf

# def fetch_stock_data(df_full_set, year, mode):
#     # Filter data for the given year
#     df_processed = df_full_set[df_full_set.Year == year]

#     # Get unique tickers
#     tickers = df_processed['tic'].unique().tolist()

#     # Define the period based on the mode
#     if mode == 'in-sample':
#         # In-sample mode: start and end within the same year
#         start_date = f'{year}-01-01'
#         end_date = f'{year}-12-31'
#     elif mode == 'forward':
#         # Forward mode: next two years
#         start_date = f'{year+1}-01-01'
#         end_date = f'{year+2}-12-31'
#     elif mode == 'backtest':
#         # Backtest mode: last 10 years, up until year-1
#         start_date = f'{year-10}-01-01'
#         end_date = f'{year-1}-12-31'
#     else:
#         raise ValueError("Mode must be 'in-sample', 'forward', or 'backtest'")

#     # Fetch historical stock data for all tickers
#     stock_data = yf.download(tickers, threads= False, start=start_date, end=end_date)

#     # Get adjusted close prices
#     close_prices = stock_data['Adj Close']

#     # Drop columns where all values are NaN
#     close_prices = close_prices.dropna(axis=1, how='all')

#     # Calculate daily returns
#     returns = close_prices.pct_change()

#     # Melt the returns DataFrame to long format for merging
#     returns_long = returns.reset_index().melt(id_vars='Date', var_name='tic', value_name='Return')



# # Merge the returns with the original dataframe



#     return returns_long


In [20]:
# def intra_industry_correlations(df, classification_column):
#     intra_matrix = []
#     industry_groups = df.groupby(classification_column)
#     intra_correlations = {}
#     for industry, group in industry_groups:
#         pivot_table = group.pivot(index='Date', columns='tic', values='Return')
#         corr_matrix = pivot_table.corr()
#         # Take the upper triangle of the correlation matrix without the diagonal
#         intra_corr = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)).stack().mean()
#         intra_correlations[industry] = intra_corr
#     return intra_correlations

In [21]:
# def cluster(target, n_clusters):
#     # Convert to a numpy array
#     X = np.array(target)

#     # Initialize and fit the k-means model
#     kmeans = KMeans(n_clusters=n_clusters, random_state=42)
#     kmeans.fit(X)

#     # Get cluster assignments
#     labels = kmeans.labels_
#     print("Cluster Labels:", labels)

#     # Get cluster centers
#     centers = kmeans.cluster_centers_
#     print("Cluster Centers:", centers)

#     wcss = np.sum((X - centers[labels])**2)
#     print("Within-Cluster Sum of Squares (WCSS):", wcss)

#     return labels


## Load relative data

#### Load item 1 data

In [4]:
file_path = "baseline_data/merged_df_from_item1.csv"
item1_df = pd.read_csv(file_path)
item1_df = item1_df.drop(['Cluster_10', 'Cluster_100'], axis=1)
item1_df.head()

print(len(item1_df['tic'].unique()))

with open("baseline_data/firm_subset.json", "r") as file:
    subset = json.load(file)

item1_df = item1_df[item1_df['tic'].isin(subset)]
len(item1_df['tic'].unique())

1197


534

#### Load return data

In [5]:
returns_long = pd.read_csv("baseline_data/return_for_merged_item1.csv")
returns_long.head()

,Date,tic,Return
0,2022-01-03,AA,NaN
1,2022-01-04,AA,-0.046885
2,2022-01-05,AA,0.017730
3,2022-01-06,AA,-0.001708
4,2022-01-07,AA,0.067066


#### Load financial variables

In [6]:
file_path = "baseline_data/variable_2021.csv"
variable_df = pd.read_csv(file_path)

#### Merge item 1 and financial variables

In [7]:
# Convert the data type
variable_df = variable_df.rename(columns={'cik': 'CIK', 'fyear': 'Year'})

variable_df['CIK'] = variable_df["CIK"].astype("int64")
variable_df['Year'] = variable_df['Year'].astype("int64")

print(variable_df['CIK'].dtype)
print(variable_df['Year'].dtype)
variable_df.head()

int64
int64


,caps,sale,at,ap,bkvlps,ceq,ch,ci,cogs,dp,CIK,Year
0,477.500,1817.100,1573.900,156.400,29.2306,1034.500,53.500,77.400,1470.300,33.100,1750,2021
1,252.989,324.140,416.197,56.242,5.8397,204.554,29.902,4.409,220.018,10.989,3197,2021
2,41.514,0.000,0.284,0.975,-0.0527,-3.011,0.045,-0.614,0.000,0.000,1961,2021
3,11.930,182.088,144.439,8.977,21.8921,77.082,4.843,13.101,113.238,4.049,2098,2021
4,16.913,2025.204,374.703,168.224,36.8280,160.386,97.825,11.888,1975.547,19.797,2178,2021


In [8]:
print(len(item1_df))
merged_df = pd.merge(item1_df, variable_df, how='left', on=['CIK', 'Year'])
print(len(merged_df))
merged_df = merged_df.dropna()
print(len(merged_df))

merged_df.head()

534
534
534


,CIK,Year,Embeddings,tic,caps,sale,at,ap,bkvlps,ceq,ch,ci,cogs,dp
0,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,784.743,2258.586,1857.712,90.090,29.7233,546.909,81.211,342.272,1679.802,149.811
1,1575515,2021,"[-0.023708730936050415, 0.01762891560792923, 0...",SFM,704.701,6099.869,2923.115,145.901,8.6387,959.876,245.287,248.873,3888.815,124.100
2,1125376,2021,"[0.04463111609220505, 0.0018805989529937506, 0...",ENSG,369.760,2627.461,2850.623,58.116,18.4955,1020.768,262.201,197.725,2161.615,55.985
3,1674335,2021,"[-0.0054583558812737465, 0.04776754230260849, ...",JELD,719.451,4771.719,3738.671,418.774,9.3378,842.218,395.596,133.769,3703.208,134.246
4,1023128,2021,"[0.007417375687509775, -0.03694348409771919, -...",LAD,58.300,22831.700,11146.900,235.400,156.8271,4626.400,174.800,1066.000,18572.700,127.300


In [9]:
# Normalization
from sklearn.preprocessing import MaxAbsScaler

scaler = MaxAbsScaler()
variable_columns = variable_df.columns.drop(['CIK', 'Year'])
merged_df[variable_columns] = pd.DataFrame(scaler.fit_transform(merged_df[variable_columns]), columns=variable_columns)
merged_df.head()


# # Normalization
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# variable_columns = variable_df.columns.drop(['cik', 'fyear'])
# variable_df[variable_columns] = pd.DataFrame(scaler.fit_transform(variable_df[variable_columns]), columns=variable_columns)
# variable_df.head()


# # Normalization
# from sklearn.preprocessing import PowerTransformer

# scaler = PowerTransformer('yeo-johnson')
# variable_columns = variable_df.columns.drop(['cik', 'fyear'])
# variable_df[variable_columns] = pd.DataFrame(scaler.fit_transform(variable_df[variable_columns]), columns=variable_columns)
# variable_df.head()

,CIK,Year,Embeddings,tic,caps,sale,at,ap,bkvlps,ceq,ch,ci,cogs,dp
0,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,0.006031,0.007853,0.002445,0.001834,0.028747,0.003288,0.003799,0.014841,0.007707,0.006553
1,1575515,2021,"[-0.023708730936050415, 0.01762891560792923, 0...",SFM,0.005416,0.021210,0.003848,0.002970,0.008355,0.005771,0.011475,0.010791,0.017843,0.005428
2,1125376,2021,"[0.04463111609220505, 0.0018805989529937506, 0...",ENSG,0.002842,0.009136,0.003752,0.001183,0.017888,0.006137,0.012267,0.008574,0.009918,0.002449
3,1674335,2021,"[-0.0054583558812737465, 0.04776754230260849, ...",JELD,0.005529,0.016592,0.004921,0.008524,0.009031,0.005063,0.018507,0.005800,0.016991,0.005872
4,1023128,2021,"[0.007417375687509775, -0.03694348409771919, -...",LAD,0.000448,0.079388,0.014673,0.004792,0.151675,0.027814,0.008178,0.046223,0.085217,0.005568


## Experiment1: Embedding only, dimension 1536

In [10]:
embedding_list = list(merged_df['Embeddings'].apply(json.loads))
len(embedding_list)

534

In [11]:
labels_10 = cluster(embedding_list, 10)
merged_df['Cluster_10'] = labels_10
labels_100 = cluster(embedding_list, 100)
merged_df['Cluster_100'] = labels_100
merged_df.head()

Cluster Labels: [8 4 2 8 3 5 8 4 8 8 7 0 4 8 8 3 5 9 8 5 8 2 1 9 8 5 2 1 3 8 5 1 4 8 5 6 4
 8 1 0 7 1 2 2 9 8 4 8 9 7 0 8 5 4 0 5 9 4 5 9 9 3 5 9 1 1 3 5 9 0 3 5 9 4
 4 2 3 8 3 5 4 9 5 1 4 4 2 1 8 5 4 1 7 8 8 4 8 3 4 8 2 7 2 9 4 8 5 8 9 9 7
 4 9 7 8 4 5 8 7 2 9 8 7 1 1 7 7 5 1 0 1 4 6 8 3 7 7 6 8 5 4 4 5 0 1 9 7 3
 3 3 8 1 3 8 2 0 3 3 3 9 3 6 9 5 8 2 9 1 5 4 3 8 7 8 8 7 3 7 6 5 4 2 8 2 5
 8 8 2 5 4 4 2 5 4 9 2 7 8 8 1 0 9 7 9 5 1 1 5 5 1 4 9 8 1 1 4 7 8 7 0 7 4
 8 7 4 2 8 1 5 7 2 2 4 3 8 5 1 9 5 5 7 1 6 6 7 7 5 6 2 4 3 8 2 0 8 4 4 5 5
 9 1 9 0 5 2 4 1 3 5 7 4 8 7 9 8 4 3 1 8 1 4 9 4 8 2 2 2 9 2 2 4 7 1 4 1 9
 4 7 8 1 5 5 4 8 8 0 3 7 8 7 7 9 8 5 7 2 2 8 6 4 5 2 8 8 2 7 5 2 3 7 7 9 4
 8 8 2 2 9 7 3 8 4 1 3 4 3 7 5 0 4 4 8 3 8 5 6 4 6 4 7 7 7 3 0 5 9 4 9 8 3
 2 1 5 9 9 9 9 7 8 1 1 8 6 9 9 8 8 9 8 7 2 4 3 7 8 8 8 3 9 8 4 4 0 9 9 5 2
 5 1 1 7 4 5 8 2 5 8 7 8 8 4 8 5 2 3 7 7 9 8 3 6 8 4 8 2 1 3 8 4 4 2 9 5 3
 2 5 3 1 8 5 1 8 4 1 8 5 8 5 0 8 4 0 5 7 3 8 8 8 7 4 5 9 2 5 5 1 8 5 3 2 9
 8 8 5 0 

,CIK,Year,Embeddings,tic,caps,sale,at,ap,bkvlps,ceq,ch,ci,cogs,dp,Cluster_10,Cluster_100
0,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,0.006031,0.007853,0.002445,0.001834,0.028747,0.003288,0.003799,0.014841,0.007707,0.006553,8,72
1,1575515,2021,"[-0.023708730936050415, 0.01762891560792923, 0...",SFM,0.005416,0.021210,0.003848,0.002970,0.008355,0.005771,0.011475,0.010791,0.017843,0.005428,4,23
2,1125376,2021,"[0.04463111609220505, 0.0018805989529937506, 0...",ENSG,0.002842,0.009136,0.003752,0.001183,0.017888,0.006137,0.012267,0.008574,0.009918,0.002449,2,28
3,1674335,2021,"[-0.0054583558812737465, 0.04776754230260849, ...",JELD,0.005529,0.016592,0.004921,0.008524,0.009031,0.005063,0.018507,0.005800,0.016991,0.005872,8,19
4,1023128,2021,"[0.007417375687509775, -0.03694348409771919, -...",LAD,0.000448,0.079388,0.014673,0.004792,0.151675,0.027814,0.008178,0.046223,0.085217,0.005568,3,34


In [31]:
df_returns = pd.merge(merged_df, returns_long, on='tic', how='inner')
print(len(df_returns['tic'].unique()))
df_returns.head()

534


,CIK,Year,Embeddings,tic,caps,sale,at,ap,bkvlps,ceq,ch,ci,cogs,dp,Cluster_10,Cluster_100,Date,Return
0,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,-0.092269,-0.065152,-0.104331,-0.043879,0.512625,-0.123576,-0.149964,0.009492,-0.037193,-0.044028,8,72,2022-01-03,NaN
1,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,-0.092269,-0.065152,-0.104331,-0.043879,0.512625,-0.123576,-0.149964,0.009492,-0.037193,-0.044028,8,72,2022-01-04,0.016987
2,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,-0.092269,-0.065152,-0.104331,-0.043879,0.512625,-0.123576,-0.149964,0.009492,-0.037193,-0.044028,8,72,2022-01-05,-0.019981
3,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,-0.092269,-0.065152,-0.104331,-0.043879,0.512625,-0.123576,-0.149964,0.009492,-0.037193,-0.044028,8,72,2022-01-06,0.001434
4,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,-0.092269,-0.065152,-0.104331,-0.043879,0.512625,-0.123576,-0.149964,0.009492,-0.037193,-0.044028,8,72,2022-01-07,-0.002704


In [32]:
intra_corrs_10 = intra_industry_correlations(df_returns, "Cluster_10")
avg_intra_corr_10 = np.nanmean(list(intra_corrs_10.values()))
print(avg_intra_corr_10)

intra_corrs_100 = intra_industry_correlations(df_returns, "Cluster_100")
avg_intra_corr_100 = np.nanmean(list(intra_corrs_100.values()))
print(avg_intra_corr_100)

0.3472796232527919
0.4172189683301795


## Experiment2: Embedding only, dimension 20

In [96]:
from sklearn.decomposition import PCA
embedding_list = list(merged_df['Embeddings'].apply(json.loads))

pca = PCA(n_components=20)
embedding_pca = pca.fit_transform(embedding_list)
embedding_pca = np.array(embedding_pca)
embedding_pca.shape

(534, 20)

In [97]:
labels_10 = cluster(embedding_pca, 10)
merged_df['Cluster_10'] = labels_10
labels_100 = cluster(embedding_pca, 100)
merged_df['Cluster_100'] = labels_100
merged_df.head()

Cluster Labels: [4 6 2 7 5 8 3 6 7 7 9 0 6 4 0 3 9 8 4 3 7 2 6 8 4 3 2 1 5 5 3 9 1 4 3 5 6
 0 9 0 4 9 2 2 8 7 1 6 8 5 0 3 7 1 0 3 9 1 9 8 8 5 7 8 9 9 5 3 8 0 5 3 8 6
 1 2 5 0 5 3 1 8 3 9 1 6 2 9 4 3 1 9 0 4 6 1 4 5 6 6 2 0 2 8 7 7 3 4 8 8 4
 6 8 4 6 6 3 3 0 2 8 7 5 9 9 8 5 3 9 0 9 1 8 4 5 9 5 7 4 3 6 1 3 0 9 8 5 5
 5 3 6 9 5 0 2 0 5 5 5 8 5 8 9 3 4 2 8 9 3 1 5 6 7 6 7 4 3 1 8 3 7 2 3 2 3
 4 4 2 3 1 1 2 5 1 8 2 5 3 4 9 0 8 5 8 3 9 9 3 3 9 1 8 7 7 7 1 5 3 7 4 5 1
 7 3 6 2 7 8 3 5 2 2 6 5 3 3 9 8 3 3 5 9 8 8 5 7 7 8 2 1 5 3 2 0 4 1 6 3 3
 8 1 8 0 5 2 1 9 5 3 5 6 7 5 8 4 6 5 9 7 9 6 8 7 6 2 2 2 8 2 2 1 7 9 1 9 8
 1 7 4 9 3 3 1 4 4 0 3 5 6 5 5 5 7 3 5 2 2 7 8 1 3 2 7 7 2 9 3 2 3 5 0 8 1
 4 7 2 2 8 7 5 4 6 8 5 1 5 5 3 0 1 1 7 5 6 3 8 1 9 1 0 5 4 5 4 3 8 6 8 6 5
 2 9 9 8 8 8 8 0 7 9 9 3 9 8 8 0 0 1 4 5 2 1 3 7 3 7 3 5 8 7 1 1 0 8 8 8 2
 9 9 9 5 6 8 4 2 3 3 4 7 3 1 6 3 2 5 5 5 8 7 5 8 3 1 6 2 9 5 7 1 1 2 8 3 5
 2 3 3 8 6 3 7 7 6 9 4 5 0 3 0 3 6 0 9 4 7 7 6 4 5 1 3 8 2 8 3 9 4 5 3 2 8
 4 6 3 0 

,CIK,Year,Embeddings,tic,caps,sale,at,ap,bkvlps,ceq,ch,ci,cogs,dp,Cluster_10,Cluster_100
0,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,0.006031,0.007853,0.002445,0.001834,0.028747,0.003288,0.003799,0.014841,0.007707,0.006553,4,40
1,1575515,2021,"[-0.023708730936050415, 0.01762891560792923, 0...",SFM,0.005416,0.021210,0.003848,0.002970,0.008355,0.005771,0.011475,0.010791,0.017843,0.005428,6,79
2,1125376,2021,"[0.04463111609220505, 0.0018805989529937506, 0...",ENSG,0.002842,0.009136,0.003752,0.001183,0.017888,0.006137,0.012267,0.008574,0.009918,0.002449,2,74
3,1674335,2021,"[-0.0054583558812737465, 0.04776754230260849, ...",JELD,0.005529,0.016592,0.004921,0.008524,0.009031,0.005063,0.018507,0.005800,0.016991,0.005872,7,12
4,1023128,2021,"[0.007417375687509775, -0.03694348409771919, -...",LAD,0.000448,0.079388,0.014673,0.004792,0.151675,0.027814,0.008178,0.046223,0.085217,0.005568,5,81


In [98]:
df_returns = pd.merge(merged_df, returns_long, on='tic', how='inner')
print(len(df_returns['tic'].unique()))
df_returns.head()

534


,CIK,Year,Embeddings,tic,caps,sale,at,ap,bkvlps,ceq,ch,ci,cogs,dp,Cluster_10,Cluster_100,Date,Return
0,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,0.006031,0.007853,0.002445,0.001834,0.028747,0.003288,0.003799,0.014841,0.007707,0.006553,4,40,2022-01-03,NaN
1,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,0.006031,0.007853,0.002445,0.001834,0.028747,0.003288,0.003799,0.014841,0.007707,0.006553,4,40,2022-01-04,0.016987
2,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,0.006031,0.007853,0.002445,0.001834,0.028747,0.003288,0.003799,0.014841,0.007707,0.006553,4,40,2022-01-05,-0.019981
3,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,0.006031,0.007853,0.002445,0.001834,0.028747,0.003288,0.003799,0.014841,0.007707,0.006553,4,40,2022-01-06,0.001434
4,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,0.006031,0.007853,0.002445,0.001834,0.028747,0.003288,0.003799,0.014841,0.007707,0.006553,4,40,2022-01-07,-0.002704


In [99]:
intra_corrs_10 = intra_industry_correlations(df_returns, "Cluster_10")
avg_intra_corr_10 = np.nanmean(list(intra_corrs_10.values()))
print(avg_intra_corr_10)

intra_corrs_100 = intra_industry_correlations(df_returns, "Cluster_100")
avg_intra_corr_100 = np.nanmean(list(intra_corrs_100.values()))
print(avg_intra_corr_100)

0.34061087781046295
0.4031728536482857


## Experiment3: Embedding + financial data, embedding dimension 10

In [115]:
from sklearn.decomposition import PCA
embedding_list = list(merged_df['Embeddings'].apply(json.loads))

pca = PCA(n_components=10)
embedding_pca = pca.fit_transform(embedding_list)
embedding_pca = np.array(embedding_pca)
embedding_pca.shape

(534, 10)

In [116]:
v_np = merged_df[variable_columns].to_numpy()

concatenated_array = np.concatenate((embedding_pca, v_np), axis=1)
concatenated_array.shape

(534, 20)

In [117]:
labels_10 = cluster(concatenated_array, 10)
merged_df['Cluster_10'] = labels_10
labels_100 = cluster(concatenated_array, 100)
merged_df['Cluster_100'] = labels_100
merged_df.head()

Cluster Labels: [0 6 7 2 8 4 2 6 2 5 1 0 3 5 0 8 9 4 2 4 2 7 1 4 2 4 7 6 8 5 2 1 6 0 2 8 5
 0 1 0 0 1 7 3 4 5 6 5 4 8 4 2 6 3 0 4 4 6 4 3 4 8 2 4 1 1 8 2 4 0 8 2 1 5
 6 7 8 0 8 0 6 1 2 1 6 3 7 1 2 2 6 1 0 5 5 6 5 8 6 5 7 0 7 4 6 2 2 0 1 4 0
 6 1 0 5 5 4 2 0 7 4 5 8 1 1 4 8 2 1 0 1 6 1 2 8 1 8 1 0 2 5 6 2 0 1 4 8 8
 8 2 5 4 8 5 7 0 8 8 8 4 9 1 4 2 5 7 4 1 2 6 8 5 8 5 5 0 2 6 1 4 5 7 2 7 2
 5 0 7 2 6 6 7 4 6 4 7 8 2 2 1 0 4 8 4 4 1 1 2 2 1 6 4 0 1 1 6 8 2 2 0 8 6
 0 4 6 7 5 1 4 3 7 7 5 8 2 2 1 4 2 2 8 1 1 1 8 0 6 9 7 6 8 2 3 0 0 6 5 2 2
 4 6 4 0 8 7 6 6 2 2 8 6 5 8 4 0 5 8 1 0 1 6 1 6 5 7 7 7 4 7 7 3 8 1 6 1 1
 6 8 0 1 2 2 6 5 0 0 2 8 5 8 8 1 5 2 8 7 7 2 1 6 2 7 5 5 7 1 2 7 2 8 0 4 6
 2 5 7 7 4 8 8 0 5 1 8 6 8 8 2 0 6 6 5 8 5 2 1 6 1 6 0 8 0 8 0 2 4 5 4 5 8
 3 1 9 4 4 1 4 0 6 1 1 5 1 4 4 0 0 1 0 8 7 6 2 5 2 5 2 8 1 5 6 6 0 4 4 4 7
 4 1 6 8 6 4 5 7 4 2 0 5 2 6 5 4 7 8 8 8 1 5 8 4 5 6 5 7 1 8 5 6 6 7 4 2 8
 9 2 2 1 5 2 1 5 5 1 0 8 0 2 3 2 5 0 4 0 8 5 5 2 8 6 2 4 7 4 2 1 0 8 2 7 1
 0 5 2 0 

,CIK,Year,Embeddings,tic,caps,sale,at,ap,bkvlps,ceq,ch,ci,cogs,dp,Cluster_10,Cluster_100
0,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,0.006031,0.007853,0.002445,0.001834,0.028747,0.003288,0.003799,0.014841,0.007707,0.006553,0,39
1,1575515,2021,"[-0.023708730936050415, 0.01762891560792923, 0...",SFM,0.005416,0.021210,0.003848,0.002970,0.008355,0.005771,0.011475,0.010791,0.017843,0.005428,6,67
2,1125376,2021,"[0.04463111609220505, 0.0018805989529937506, 0...",ENSG,0.002842,0.009136,0.003752,0.001183,0.017888,0.006137,0.012267,0.008574,0.009918,0.002449,7,23
3,1674335,2021,"[-0.0054583558812737465, 0.04776754230260849, ...",JELD,0.005529,0.016592,0.004921,0.008524,0.009031,0.005063,0.018507,0.005800,0.016991,0.005872,2,73
4,1023128,2021,"[0.007417375687509775, -0.03694348409771919, -...",LAD,0.000448,0.079388,0.014673,0.004792,0.151675,0.027814,0.008178,0.046223,0.085217,0.005568,8,34


In [118]:
df_returns = pd.merge(merged_df, returns_long, on='tic', how='inner')
print(len(df_returns['tic'].unique()))
df_returns.head()

534


,CIK,Year,Embeddings,tic,caps,sale,at,ap,bkvlps,ceq,ch,ci,cogs,dp,Cluster_10,Cluster_100,Date,Return
0,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,0.006031,0.007853,0.002445,0.001834,0.028747,0.003288,0.003799,0.014841,0.007707,0.006553,0,39,2022-01-03,NaN
1,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,0.006031,0.007853,0.002445,0.001834,0.028747,0.003288,0.003799,0.014841,0.007707,0.006553,0,39,2022-01-04,0.016987
2,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,0.006031,0.007853,0.002445,0.001834,0.028747,0.003288,0.003799,0.014841,0.007707,0.006553,0,39,2022-01-05,-0.019981
3,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,0.006031,0.007853,0.002445,0.001834,0.028747,0.003288,0.003799,0.014841,0.007707,0.006553,0,39,2022-01-06,0.001434
4,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,0.006031,0.007853,0.002445,0.001834,0.028747,0.003288,0.003799,0.014841,0.007707,0.006553,0,39,2022-01-07,-0.002704


In [119]:
intra_corrs_10 = intra_industry_correlations(df_returns, "Cluster_10")
avg_intra_corr_10 = np.nanmean(list(intra_corrs_10.values()))
print(avg_intra_corr_10)

intra_corrs_100 = intra_industry_correlations(df_returns, "Cluster_100")
avg_intra_corr_100 = np.nanmean(list(intra_corrs_100.values()))
print(avg_intra_corr_100)

0.32369311944700674
0.4222348178665087


## Control experiment: All firms cluster to 1 category

In [66]:
merged_df['Cluster_1'] = np.ones(len(merged_df), dtype="int64")
merged_df.head()

,CIK,Year,Embeddings,tic,caps,sale,at,ap,bkvlps,ceq,ch,ci,cogs,dp,Cluster_10,Cluster_100,Cluster_1
0,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,-0.092269,-0.065152,-0.104331,-0.043879,0.512625,-0.123576,-0.149964,0.009492,-0.037193,-0.044028,1,55,1
1,1575515,2021,"[-0.023708730936050415, 0.01762891560792923, 0...",SFM,-0.106419,0.176978,-0.083594,-0.041736,-0.077995,-0.088745,-0.097851,-0.030230,0.147011,-0.068653,1,99,1
2,1125376,2021,"[0.04463111609220505, 0.0018805989529937506, 0...",ENSG,-0.165628,-0.041900,-0.085005,-0.045106,0.198113,-0.083609,-0.092479,-0.051983,0.002984,-0.133889,1,83,1
3,1674335,2021,"[-0.0054583558812737465, 0.04776754230260849, ...",JELD,-0.103811,0.093260,-0.067720,-0.031261,-0.058412,-0.098668,-0.050112,-0.079183,0.131534,-0.058936,1,99,1
4,1023128,2021,"[0.007417375687509775, -0.03694348409771919, -...",LAD,-0.220687,1.231641,0.076474,-0.038301,4.073046,0.220501,-0.120239,0.317287,1.371462,-0.065588,0,12,1


In [67]:
df_returns = pd.merge(merged_df, returns_long, on='tic', how='inner')
print(len(df_returns['tic'].unique()))
df_returns.head()

534


,CIK,Year,Embeddings,tic,caps,sale,at,ap,bkvlps,ceq,ch,ci,cogs,dp,Cluster_10,Cluster_100,Cluster_1,Date,Return
0,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,-0.092269,-0.065152,-0.104331,-0.043879,0.512625,-0.123576,-0.149964,0.009492,-0.037193,-0.044028,1,55,1,2022-01-03,NaN
1,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,-0.092269,-0.065152,-0.104331,-0.043879,0.512625,-0.123576,-0.149964,0.009492,-0.037193,-0.044028,1,55,1,2022-01-04,0.016987
2,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,-0.092269,-0.065152,-0.104331,-0.043879,0.512625,-0.123576,-0.149964,0.009492,-0.037193,-0.044028,1,55,1,2022-01-05,-0.019981
3,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,-0.092269,-0.065152,-0.104331,-0.043879,0.512625,-0.123576,-0.149964,0.009492,-0.037193,-0.044028,1,55,1,2022-01-06,0.001434
4,1704715,2021,"[0.014074714854359627, 0.006938479840755463, 0...",AMR,-0.092269,-0.065152,-0.104331,-0.043879,0.512625,-0.123576,-0.149964,0.009492,-0.037193,-0.044028,1,55,1,2022-01-07,-0.002704


In [68]:
intra_corrs_1 = intra_industry_correlations(df_returns, "Cluster_1")
avg_intra_corr_1 = np.nanmean(list(intra_corrs_1.values()))
print(avg_intra_corr_1)

0.30203190501983
